# Prophet

In [1]:
# Imports
import numpy as np
import pandas as pd
from prophet import Prophet
from sklearn.metrics import r2_score

/home/jose/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Carregar os dados de treinamento
train_data = pd.read_excel('../baseDeDados/dataBaseTratadaDistancia.xlsx')

# Carregar os dados de teste
test_data = pd.read_excel('../baseDeDados/dataBaseTeste.xlsx')

In [13]:
# Preparar os dados de treinamento para Prophet
train_prophet_data = train_data[['order_purchase_timestamp', 'delivery_time']].copy()
train_prophet_data.rename(columns={'order_purchase_timestamp': 'ds', 'delivery_time': 'y'}, inplace=True)
train_prophet_data.dropna(inplace=True)

# Verificar e ajustar valores antes da transformação logarítmica
train_prophet_data['y'] = train_prophet_data['y'].apply(lambda x: max(x, 11))

# Subtrair 13 dos valores de 'delivery_time' antes da transformação logarítmica
train_prophet_data['y'] = np.log1p(train_prophet_data['y'] - 11)


# Criar e treinar o modelo Prophet com ajustes de sazonalidade
model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model.fit(train_prophet_data)

# Fazer previsões nos dados de treinamento
forecast = model.predict(train_prophet_data)

# Aplicar a transformação inversa (exponencial) aos valores previstos e adicionar 13
forecast['yhat'] = np.expm1(forecast['yhat']) + 11

# Preparar os dados de teste para Prophet
test_prophet_data = test_data[['order_purchase_timestamp']].copy()
test_prophet_data.rename(columns={'order_purchase_timestamp': 'ds'}, inplace=True)

# Fazer previsões nos dados de teste
forecast_test = model.predict(test_prophet_data)

# Aplicar a transformação inversa (exponencial) aos valores previstos e adicionar 13
forecast_test['yhat'] = np.expm1(forecast_test['yhat']) + 11

# Adicionar previsões ao dataset de teste
test_data['previsoes'] = forecast_test['yhat']


20:02:59 - cmdstanpy - INFO - Chain [1] start processing
20:03:18 - cmdstanpy - INFO - Chain [1] done processing


In [14]:
# Mostrar os resultados
print(test_data[['order_id', 'previsoes']].head())

                           order_id  previsoes
0  cee3292f46ede6ea1dfabfcb200fcf47  13.871654
1  50dca53ca33b739bef09e7933e8b380e  13.974268
2  8087ec71e393d4dc6fc48041fe63cd51  12.958998
3  e6b6557ce111de79b31cc857f20ba212  13.249623
4  0b09c5e4c2512f627190ac55a78c35a3  12.981256


In [15]:
# Alinhar os índices dos dados reais e previstos para garantir o mesmo tamanho
aligned_data = pd.merge(train_prophet_data, forecast[['ds', 'yhat']], on='ds', how='inner')

# Calcular o R^2 usando os valores reais e previstos
r2 = r2_score(aligned_data['y'], aligned_data['yhat'])
print(f'R^2: {r2}')

R^2: -146.66792177953047


In [16]:
# Enviando para um dataFrame do Kaggle
dfKaggle = test_data[['order_id', 'previsoes']]

# Salvar os resultados
dfKaggle.to_csv('../resultadoPrevisoes/previsaoProphet.csv', index=False)

In [17]:
dfKaggle

,order_id,previsoes
0,cee3292f46ede6ea1dfabfcb200fcf47,13.871654
1,50dca53ca33b739bef09e7933e8b380e,13.974268
2,8087ec71e393d4dc6fc48041fe63cd51,12.958998
3,e6b6557ce111de79b31cc857f20ba212,13.249623
4,0b09c5e4c2512f627190ac55a78c35a3,12.981256
...,...,...
28147,81cedc4d2132eb6f512354166e51b187,10.045431
28148,95811675a89b66659996da5285a021db,10.045323
28149,25bd2ad9d282f59e4e6a048ef9bf214f,10.045323
28150,9f9ac312604229686c04260955495c70,10.045290
